In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# **Trabalho de conclusao - Data Mining e Machine Learning II**

Sera apresentado a seguir estudo para base de emprestimo HMEQ.CSV, com analise exploratoria dos dados e modelo preditivo em random forest com e sem cross validation.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Importando dataset
df= pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')
df.shape


In [ ]:
df.head()

In [ ]:
df.info()

Temos algumas colunas sem registro(NaN),principalmente a coluna de Debtinc com 1267 valores faltantes.

# Analise Exploratória

In [ ]:
graf1 = sns.barplot(x=df.BAD, y=df.BAD, data=df, estimator=lambda x: len(x) / len(df) * 100)
graf1.set(ylabel="Percent")

20% dos Clientes sao maus pagadores 

In [ ]:
sns.distplot(df.LOAN.dropna())
plt.title('Distribuição de quantidade de emprestimo')
plt.xlabel('Valor')
plt.ylabel('Quantidade')


In [ ]:
sns.distplot(df.MORTDUE.dropna())
plt.title('Distribição do valor do empréstimo ')
plt.ylabel('Quantidade')
plt.xlabel('Valor da divida')

In [ ]:
sns.distplot(df.VALUE.dropna())
plt.title('Distribuição do valor do imovel')
plt.xlabel('Valor')
plt.ylabel('Quantidade')

In [ ]:
plt.title('Distribuiçao de anos de trabalho')
sns.distplot(df.YOJ.dropna())
plt.xlabel('Anos trabalhaos')
plt.ylabel('Quantidade')

In [ ]:
a = df.REASON.copy()
a[pd.isnull(a)==True] = 'Missing'
sns.countplot(a, hue = df.BAD)
plt.title('Status de Emprestimo para pagar outro emprestimo')
plt.xlabel('Motivo')
plt.ylabel('Quantidade')

In [ ]:
b = df.JOB.copy()
b[pd.isnull(b)==True] = 'Missing'
sns.countplot(b, hue= df.BAD)
plt.title('Emprestimo por Cargo')
plt.ylabel('Quantidade')
plt.xlabel('Cargo')

A marioria dos emprestimos sao para pagamento de outras dividas, o maior volume de pessos que procuram emprestimos tem o cargo classificados como outros

In [ ]:
def dist_boxplot(x, **kwargs):
    ax = sns.distplot(x, hist_kws=dict(alpha=0.2))
    ax2 = ax.twinx()
    sns.boxplot(x=x, ax=ax2)
    ax2.set(ylim=(-5, 5))

g = sns.FacetGrid(df, col="BAD")
g1 = g.map(dist_boxplot, 'CLAGE', data = df)

g = sns.FacetGrid(df, col="BAD")
g2 = g.map(dist_boxplot, 'NINQ', data = df)

g = sns.FacetGrid(df, col="BAD")
g3 = g.map(dist_boxplot, 'CLNO', data = df)

g = sns.FacetGrid(df, col="BAD")
g4 = g.map(dist_boxplot, 'DEBTINC', data = df)

As variaveis CLAGE, NINQ, CLNO e DEBTINC parecem variar bastante em relação à variável BAD e podem servir como possíveis preditores.

**Substituindo valores faltantes em Dectinc

In [ ]:
# input de valores na varial DEBTINC pela media de acordo com a descrição da coluna REASON
df.loc[(df.REASON == 'HomeImp') & (pd.isnull(df.DEBTINC) == True),'DEBTINC'] = np.mean(df.loc[df.REASON == 'HomeImp','DEBTINC'])
df.loc[(df.REASON == 'DebtCon') & (pd.isnull(df.DEBTINC) == True),'DEBTINC'] = np.mean(df.loc[df.REASON == 'DebtCon','DEBTINC'])
df.loc[(pd.isnull(df.REASON) == True) & (pd.isnull(df.DEBTINC) == True),'DEBTINC'] = np.mean(df.loc[pd.isnull(df.REASON) == True,'DEBTINC'])
df.info()

In [ ]:
#input de 0 para valores faltantes na coluna de credito inadimplente 
df.loc[pd.isnull(df.DELINQ)==True, 'DELINQ'] = 0
df.loc[pd.isnull(df.JOB)==True, 'JOB'] = 'Missing'

In [ ]:
df.isnull().sum()

In [ ]:
df = df.fillna(0)

In [ ]:
df.shape

In [ ]:
df.info()

## Criando Modelo de Random Forest

In [ ]:

import sklearn

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import scikitplot as skplt
from sklearn.model_selection import cross_val_score

Criando um novo dataframe para o modelo de random forest com inclusao de colunas dummy

In [ ]:
df_rf = pd.get_dummies(df, df.dtypes[(df.dtypes==np.object) | (df.dtypes=='category')].index.values, drop_first=True)
df_rf.head()

Separado o novo dataframe em traino e teste

In [ ]:
train, test = train_test_split(df_rf, test_size=0.20, random_state=42)

separando o base treino em treino e validação

In [ ]:
train, valid = train_test_split(train, test_size=0.20, random_state=42)

train.shape, valid.shape, test.shape

In [ ]:
df_rf.info()

### **Incio do modelo de Random Forest**

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, oob_score=True, n_estimators = 150, random_state=150)

In [ ]:
feats = [c for c in df_rf.columns if c not in ['BAD']]
feats

In [ ]:
rf.fit(train[feats],train['BAD'])

In [ ]:
# Prevendo os dados de validação
preds_val = rf.predict(valid[feats])


In [ ]:
accuracy_score(valid['BAD'], preds_val)

In [ ]:
preds_test = rf.predict(test[feats])

preds_test

In [ ]:
accuracy_score(test['BAD'], preds_test)

In [ ]:
skplt.metrics.plot_confusion_matrix(test['BAD'], preds_test)

Apos a primeira tentativa usando o modelo de random forest, é possivel observar uma acuracia de 91% considerado muito boa para o modelo.

## Revendo Random Forest utilizando Cross Validation

In [ ]:
train, test = train_test_split(df_rf, test_size=0.20, random_state=42)
train.shape, test.shape

In [ ]:
scores = cross_val_score(rf, train[feats], train['BAD'], n_jobs=-1, cv=5)

scores, scores.mean()

In [ ]:
scores = cross_val_score(rf, test[feats], test['BAD'], n_jobs=-1, cv=5)

scores, scores.mean()

Apos a realização do random foreste usando o metodo de validação cruzada de 5 conjuntos, obtivemos 92,4% de acuracia na base de treino,1% melhor que o primeito modelo de random forest testado anteriormente. Quando aplicando o mesmo modelo na base de teste tivemos o resultado de 89,3 que ainda assim é condiderado um bom modelo.